<a id=top-page></a>
# `pykitPIV` demo: Transform images

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size="3"><strong>Table of contents:</strong></font>
<br>
<ol>
    <li><a href="#upload">Upload the saved datasets</a></li>
</ol>
<p></p>
</div>
<br>

***

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import torch
import cmcrameri.cm as cmc
from pykitPIV import Image, Postprocess, Particle, FlowField

In [ ]:
save_images = False
figsize = (10,10)
filename = None

<a id=upload></a>
***

## Post-process saved dataset

[Go to the top](#top-page)

The standalone import of previously saved PIV image pairs and the associated targets can be performed following this minimal example:

In [ ]:
image = Image()

In [ ]:
images_tensor_dic = image.upload_from_h5(filename='../docs/data/pykitPIV-dataset-10-PIV-pairs-256-by-256.h5')
images_tensor = images_tensor_dic['I']
images_tensor.shape

### Pass an image tensor of size $(N, 2, H, W)$

In [ ]:
postprocess = Postprocess(images_tensor)

Check that the image pair has been passed and not just one image:

In [ ]:
postprocess.image_pair

First, we add Gaussian noise to the images:

In [ ]:
postprocess.add_gaussian_noise(loc=0.0, 
                               scale=(100,200))

In [ ]:
plt = postprocess.plot(original=False,
                 idx=2,
                 instance=1,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 figsize=figsize)

plt.colorbar()

In [ ]:
np.min(postprocess.processed_image_tensor)

Second, we perform a log-transformation of the images with the noise added:

In [ ]:
postprocess.log_transform_images(addition=1000)

In [ ]:
postprocess.processed_image_tensor.shape

In [ ]:
plt = postprocess.plot(original=False,
                 idx=0,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 figsize=figsize);

plt.colorbar()

### Pass an image tensor of size $(N, H, W)$

In [ ]:
postprocess = Postprocess(images_tensor[:,0,:,:])

Check that single image instances, $I_1$, have been passed:

In [ ]:
postprocess.image_pair

Perform a log-transformation of the images:

In [ ]:
postprocess.log_transform_images(addition=1000)

In [ ]:
postprocess.processed_image_tensor.shape

In [ ]:
plt = postprocess.plot(original=False,
                 idx=0,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 figsize=figsize)

plt.colorbar()

Add Gaussian noise to the log-transformed images:

In [ ]:
postprocess.add_gaussian_noise(loc=0.0, 
                               scale=(0.1,0.2))

In [ ]:
plt = postprocess.plot(original=False,
                 idx=2,
                 instance=1,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 figsize=figsize)

plt.colorbar()

***

## Post-process on-the-fly user-created dataset

***

## Add shot noise and Gaussian noise to images

[Go to the top](#top-page)

In [ ]:
n_images = 1
image_size = (50, 50)
size_buffer = 0
figsize=(4,4)

In [ ]:
particles = Particle(n_images, 
                     size=image_size, 
                     size_buffer=size_buffer,
                     diameters=4,
                     densities=0.05,
                     diameter_std=0,
                     seeding_mode='random', 
                     random_seed=100)

In [ ]:
image = Image(random_seed=100)

In [ ]:
image.add_particles(particles)

In [ ]:
image.add_reflected_light(exposures=(0.99,0.99),
                          maximum_intensity=2**16-1,
                          laser_beam_thickness=1,
                          laser_over_exposure=1,
                          laser_beam_shape=0.95,
                          alpha=1/8)

In [ ]:
plt = image.plot(0, 
                 with_buffer=True,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 origin='lower',
                 figsize=figsize)

plt.colorbar()

In [ ]:
images_tensor = image.images_I1[:,0,:,:]

In [ ]:
images_tensor.shape

In [ ]:
postprocess = Postprocess(images_tensor, 
                          random_seed=100)

First, we add Gaussian noise to the images:

In [ ]:
postprocess.add_shot_noise(strength=1,
                           clip=2**16-1)

In [ ]:
plt = postprocess.plot(original=True,
                 idx=0,
                 instance=1,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='Greys_r',
                 figsize=figsize)

plt.colorbar()

In [ ]:
plt = postprocess.plot(original=False,
                       idx=0,
                       instance=1,
                       xlabel='Width [px]',
                       ylabel='Height [px]',
                       cmap='Greys_r',
                       vmin=0,
                       vmax=2**16-1,
                       figsize=figsize)

plt.colorbar()

In [ ]:
np.min(postprocess.processed_image_tensor)

In [ ]:
np.max(postprocess.processed_image_tensor)

Add Gaussian noise atop the shot noise:

In [ ]:
postprocess.add_gaussian_noise(loc=0.0,
                               scale=10000,
                               clip=2**16-1)

In [ ]:
plt = postprocess.plot(original=False,
                       idx=0,
                       instance=1,
                       xlabel='Width [px]',
                       ylabel='Height [px]',
                       cmap='Greys_r',
                       figsize=figsize)

plt.colorbar()

In [ ]:
np.min(postprocess.processed_image_tensor)

In [ ]:
np.max(postprocess.processed_image_tensor)

In [ ]:
figsize=(6,3)

In [ ]:
image = Image(random_seed=100)

In [ ]:
flowfield = FlowField(1,
                      size=(30,30),
                      size_buffer=0,
                      random_seed=100)

In [ ]:
flowfield.generate_checkered_velocity_field(displacement=(0,2),
                                            m=8,
                                            n=8,
                                            rotation=None)

In [ ]:
image.add_flowfield(flowfield)

In [ ]:
image.plot_field(0,
                  with_buffer=True,
                  xlabel='Width [px]',
                  ylabel='Height [px]',
                  title=('Example random velocity component $u$', 'Example random velocity component $v$'),
                  cmap=cmc.oslo_r,
                  figsize=figsize);

In [ ]:
image_tensor = flowfield.velocity_field[0][0][None,:,:]
image_tensor.shape

In [ ]:
postprocess = Postprocess(image_tensor)

In [ ]:
log_transformed_images_tensor = postprocess.log_transform_images(addition=10)

In [ ]:
postprocess.plot(False,
                 0,
                 instance=2,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='viridis',
                 cbar=True,
                 figsize=figsize);

In [ ]:
postprocess.plot(True,
                 0,
                 instance=2,
                 xlabel='Width [px]',
                 ylabel='Height [px]',
                 cmap='viridis',
                 cbar=True,
                 figsize=figsize);

In [ ]:
filters = np.ones((5,5))

In [ ]:
filters[0,:] = 1
filters[1,:] = 0
filters[3,:] = 0
filters[4,:] = 1

In [ ]:
filters

In [ ]:
from torch.nn import Conv2d
import torch.nn.functional as F

In [ ]:
filtered_image_tensor = F.conv2d(torch.from_numpy(image_tensor), 
                                                   torch.from_numpy(filters[None,None,:,:]), 
                                                   padding='same')

In [ ]:
Hin = 10

In [ ]:
x = torch.rand(1,3,Hin,Hin)
conv = Conv2d(3, 1000, 1, padding=(1,1), stride=(1,1), dilation=(1,1))
output = conv(x)
output.shape

In [ ]:
conv.padding

In [ ]:
conv_grouped.dilation

In [ ]:
conv_grouped.kernel_size

In [ ]:
conv_grouped.stride

In [ ]:
Hout = int(np.floor((Hin - (2 * conv_grouped.padding[0]) - conv_grouped.dilation[0] * (conv_grouped.kernel_size[0] - 1) - 1) / (conv_grouped.stride[0]) + 1))
Hout

In [ ]:
conv_grouped.weight.shape

In [ ]:
conv_grouped.bias.shape

In [ ]:
from diff_conv2d import functional

In [ ]:
torch.set_default_dtype(torch.float64)
B_, Ci_, H_, W_ = 8, 4, 500, 300
n_ = 5
p_ = n_ // 2
Co_, groups_ = 1, 1
img_ = torch.rand(B_, Ci_, H_, W_)
kernel_ = torch.rand(Co_, Ci_ // groups_, n_, n_)
model = functional.DiffConv2d(kernel_size=n_)
res_s = model.conv2d(img_, kernel_, groups=groups_, optimized_for='speed')
res_m = model.conv2d(img_, kernel_, groups=groups_, optimized_for='memory')
print(res_s.shape)
print((res_s - res_m).abs().max())

In [ ]:
plt.imshow(filtered_image_tensor[0,:,:])
plt.colorbar()

In [ ]:
plt.imshow(image_tensor[0,:,:])
plt.colorbar()

In [ ]:
plt.imshow(input_tensor[0,:,:]/torch.max(input_tensor[0,:,:]) - filtered_image_tensor[0,:,:]/torch.max(filtered_image_tensor[0,:,:]), 
           cmap='coolwarm')

plt.colorbar();

In [ ]:
from diff_conv2d import functional
from diff_conv2d import layers
from diff_conv2d import maths

In [ ]:
from diff_conv2d.functional import DiffConv2d

diff = DiffConv2d(kernel_size=5)  # kernel_size can be 3, 5, or 7
output = diff.conv2d(torch.from_numpy(image_tensor[None,:,:,:]).float(), torch.from_numpy(filters[None,None,:,:]).float())

In [ ]:
output.shape

In [ ]:
plt.imshow(output[0,0,:,:])
plt.colorbar()

In [ ]:
plt.imshow(input_tensor[0,:,:]/torch.max(input_tensor[0,:,:]) - output[0,0,:,:]/torch.max(output[0,0,:,:]), 
           cmap='coolwarm', vmin=-0.3, vmax=0.3)

plt.colorbar();

In [ ]:
m = torch.nn.Conv2d(1, 1, 3, stride=1, padding='same', padding_mode='zeros', bias=False)
output = m(input_tensor.float())

In [ ]:
plt.imshow(filtered_image_tensor[0,0,:,:]/torch.max(filtered_image_tensor[0,0,:,:]))
plt.colorbar()

In [ ]:
plt.imshow(input_tensor[0,0,:,:]/torch.max(input_tensor[0,0,:,:]))
plt.colorbar()

***

© K. Zdybał, C. Mucignat, I. Lunati (2025)